# Self-refine

In [1]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict]):
        client = OpenAI(api_key='YOUR DEEPSEEK KEY', base_url="https://api.deepseek.com")
        response = client.chat.completions.create(
                model = 'deepseek-chat',
                temperature = 0,
                messages = messages
        )

        return response

## 1st time interacting with Deepseek

In [2]:
# reference https://github.com/andrewyng/translation-agent

source_text = """
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.
"""

In [3]:
## system prompt
first_translate_system_message = f"You are an expert linguist, specializing in translation from English to Chinese."

## user prompt

first_user_translation_prompt = f"""This is an English to Chinese translation, please provide the Chinese translation for this text. \
Do not provide any explanations or text apart from the translation.
English: {source_text}

Chinese:"""

## 将 source text 作为参数传入，构造好完整的输入
first_user_prompt = first_user_translation_prompt.format(source_text=source_text)

print(first_user_prompt)

This is an English to Chinese translation, please provide the Chinese translation for this text. Do not provide any explanations or text apart from the translation.
English: 
Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be applied to AI applications, not to general-purpose AI technology.


Chinese:


In [4]:
messages = [
    {"role":"system", "content": first_translate_system_message},
    {"role":"user", "content": first_user_prompt}
]

first_response = ask(messages=messages)

translation_1 = first_response.choices[0].message.content
print(translation_1)

上周，我在美国国会大厦的一次活动中谈到了人工智能和监管问题，该活动吸引了立法界和企业界的领袖。我对开源社区在抵御可能扼杀创新的法规方面取得的进展感到鼓舞。但开源的反对者正在不断转变他们的论点，最新的担忧集中在开源对国家安全的影响上。我希望我们都能继续保护开源！

根据我与立法者的对话，我对美国联邦政府在切实把握人工智能风险方面取得的进展感到鼓舞。需要明确的是，监管措施是必要的。但它们应该针对人工智能应用，而不是通用的人工智能技术。


## 2ed time interacting with Deepseek

In [5]:
reflect_system_prompt = "You are an expert linguist specializing in translation from English to Chinese. \
You will be provided with a source text and its translation and your goal is to improve the translation."

In [6]:
reflect_prompt = f"""Your task is to carefully read a source text and a translation from English to Chinese, and then give constructive criticism and helpful suggestions to improve the translation. \

The source text and initial translation, delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT> and <TRANSLATION></TRANSLATION>, are as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

When writing suggestions, pay attention to whether there are ways to improve the translation's \n\
(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),\n\
(ii) fluency (by applying Chinese grammar, spelling and punctuation rules, and ensuring there are no unnecessary repetitions),\n\
(iii) style (by ensuring the translations reflect the style of the source text and takes into account any cultural context),\n\
(iv) terminology (by ensuring terminology use is consistent and reflects the source text domain; and by only ensuring you use equivalent idioms Chinese).\n\

Write a list of specific, helpful and constructive suggestions for improving the translation.
Each suggestion should address one specific part of the translation.
Output only the suggestions and nothing else."""

feedback_user_prompt = reflect_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
    )

print(feedback_user_prompt)

Your task is to carefully read a source text and a translation from English to Chinese, and then give constructive criticism and helpful suggestions to improve the translation. 
The source text and initial translation, delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT> and <TRANSLATION></TRANSLATION>, are as follows:

<SOURCE_TEXT>

Last week, I spoke about AI and regulation at the U.S. Capitol at an event that was attended by legislative and business leaders. I’m encouraged by the progress the open source community has made fending off regulations that would have stifled innovation. But opponents of open source are continuing to shift their arguments, with the latest worries centering on open source's impact on national security. I hope we’ll all keep protecting open source!

Based on my conversations with legislators, I’m encouraged by the progress the U.S. federal government has made getting a realistic grasp of AI’s risks. To be clear, guardrails are needed. But they should be appli

In [7]:
message2 = [
    {"role":"system", "content": reflect_system_prompt},
    {"role":"user", "content": feedback_user_prompt}
]

response = ask(messages=message2)

reflection = response.choices[0].message.content
print(reflection)

1. 将“上周，我在美国国会大厦的一次活动中谈到了人工智能和监管问题”修改为“上周，我在美国国会大厦的一次活动中就人工智能及其监管问题发表了演讲”，以更准确地传达原文的演讲行为。
2. 将“该活动吸引了立法界和企业界的领袖”修改为“该活动汇聚了立法界和企业界的领袖”，以提高表达的流畅性和自然度。
3. 将“我对开源社区在抵御可能扼杀创新的法规方面取得的进展感到鼓舞”修改为“我对开源社区在抵御可能扼杀创新的法规方面所取得的进展感到鼓舞”，增加“所”字以增强语句的完整性。
4. 将“但开源的反对者正在不断转变他们的论点”修改为“但反对开源的人士正在不断转变他们的论点”，以更准确地表达原文的反对者身份。
5. 将“最新的担忧集中在开源对国家安全的影响上”修改为“最新的担忧聚焦于开源对国家安全的影响”，以提高表达的精炼度和流畅性。
6. 将“我希望我们都能继续保护开源！”修改为“我期望我们都能继续保护开源！”，以更符合原文的语气。
7. 将“根据我与立法者的对话”修改为“基于我与立法者的交流”，以更准确地传达原文的交流行为。
8. 将“我对美国联邦政府在切实把握人工智能风险方面取得的进展感到鼓舞”修改为“我对美国联邦政府在切实把握人工智能风险方面所取得的进展感到鼓舞”，增加“所”字以增强语句的完整性。
9. 将“需要明确的是，监管措施是必要的”修改为“需要明确的是，监管措施是不可或缺的”，以更准确地传达原文的强调语气。
10. 将“但它们应该针对人工智能应用，而不是通用的人工智能技术”修改为“但它们应针对人工智能的具体应用，而非通用的人工智能技术”，以提高表达的准确性和流畅性。


## 3rd time interacting with Deepseek

In [8]:
# system prompt
refiner_system_prompt = f"You are an expert linguist, specializing in translation editing from English to Chinese."

# user content
refined_translate_user_prompt = f"""Your task is to carefully read, then edit, a translation from English to Chinese, taking into
account a list of expert suggestions and constructive criticisms.

The source text, the initial translation, and the expert linguist suggestions are delimited by XML tags <SOURCE_TEXT></SOURCE_TEXT>, <TRANSLATION></TRANSLATION> and <EXPERT_SUGGESTIONS></EXPERT_SUGGESTIONS> \
as follows:

<SOURCE_TEXT>
{source_text}
</SOURCE_TEXT>

<TRANSLATION>
{translation_1}
</TRANSLATION>

<EXPERT_SUGGESTIONS>
{reflection}
</EXPERT_SUGGESTIONS>

Please take into account the expert suggestions when editing the translation. Edit the translation by ensuring:

(i) accuracy (by correcting errors of addition, mistranslation, omission, or untranslated text),
(ii) fluency (by applying Chinese grammar, spelling and punctuation rules and ensuring there are no unnecessary repetitions), \
(iii) style (by ensuring the translations reflect the style of the source text)
(iv) terminology (inappropriate for context, inconsistent use), or
(v) other errors.

Output only the new translation and nothing else."""

refined_user_prompt = refined_translate_user_prompt.format(
        source_text=source_text,
        translation_1=translation_1,
        reflection=reflection,
    )

message3 = [
    {"role":"system", "content": refiner_system_prompt},
    {"role":"user", "content": refined_user_prompt}
]

response = ask(messages=message3)

translation_2 = response.choices[0].message.content
print(translation_2)


上周，我在美国国会大厦的一次活动中就人工智能及其监管问题发表了演讲，该活动汇聚了立法界和企业界的领袖。我对开源社区在抵御可能扼杀创新的法规方面所取得的进展感到鼓舞。但反对开源的人士正在不断转变他们的论点，最新的担忧聚焦于开源对国家安全的影响。我期望我们都能继续保护开源！

基于我与立法者的交流，我对美国联邦政府在切实把握人工智能风险方面所取得的进展感到鼓舞。需要明确的是，监管措施是不可或缺的。但它们应针对人工智能的具体应用，而非通用的人工智能技术。
